# Prep

## Libraries and paths

In [109]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.stats.weightstats import DescrStatsW
import geopandas as gpd
import folium

In [110]:
main_path=os.path.dirname(os.path.abspath(os.getcwd())) # main dir
data_path = os.path.join(main_path, 'data', 'processed') # data dir
censo_path=os.path.join(data_path,'censo') #censo dir
shp_path=os.path.join(data_path,'shp') # shp dir

In [111]:
censo_ent_path=os.path.join(censo_path,'censo_ent_tidy_data.csv') # censo_ent
censo_mun_path=os.path.join(censo_path,'censo_mun_tidy_data.csv') # censo_mun
enco_path=os.path.join(data_path,'enco','enco_processed_tidy.csv') # enco
enigh_path=os.path.join(data_path,'enigh','enigh_processed_tidy.csv') # enigh
shp_ent_path=os.path.join(shp_path,'shp_ent_tidy_data.shp') # shp_ent
shp_mun_path=os.path.join(shp_path,'shp_mun_tidy_data.shp') # shp_mun

## File read

In [112]:
df_enigh = pd.read_csv(enigh_path)
df_enco = pd.read_csv(enco_path,dtype={'i_per': 'string'})
df_cve_ent = pd.read_csv(censo_ent_path, encoding='utf-8')
df_cve_mun = pd.read_csv(censo_mun_path, encoding='utf-8')
gdf_ent = gpd.read_file(shp_ent_path)
gdf_mun = gpd.read_file(shp_mun_path)

# ENIGH

## Calculo de gini/deciles

### Funciones para calcular

In [ ]:
def calcular_gini_y_deciles_ent(df):
    # Lista para almacenar los resultados por estado y año
    resultados = []

    # Agrupamos por 'year' y 'entidad'
    for (year, ent), grupo in df.groupby(['year', 'ent']):

        # Calcular el total de hogares en el grupo
        total_hogares = grupo['factor'].sum()
        if total_hogares == 0:
            continue  # Omitir el grupo si no hay hogares

        # Ordenar el grupo por ingreso y calcular el tamaño del decil
        grupo = grupo.sort_values(by='ing_cor').reset_index(drop=True)
        tam_dec = int(total_hogares // 10)  # Tamaño del decil truncado

        # Sumar acumulativamente el factor para dividir en deciles
        grupo['ACUMULA'] = grupo['factor'].cumsum()
        grupo['DECIL'] = pd.cut(grupo['ACUMULA'], bins=[0] + [tam_dec * i for i in range(1, 11)], labels=range(1, 11), include_lowest=True)

        # Calcular el ingreso promedio y el número de hogares por decil, solo si el factor > 0
        ingresos_por_decil = grupo.groupby('DECIL', observed=True).apply(
            lambda x: np.average(x['ing_cor'], weights=x['factor']) if x['factor'].sum() > 0 else 0, include_groups=False
        )
        hogares_por_decil = grupo.groupby('DECIL', observed=True)['factor'].sum()

        # Crear tabla de deciles para calcular el Gini
        tabla_deciles = pd.DataFrame({
            'hogares': hogares_por_decil,
            'ingreso_promedio': ingresos_por_decil
        }).reset_index()

        # Calcular el Gini usando ingresos promedio por decil ponderados por el número de hogares
        ingresos = tabla_deciles['ingreso_promedio'].values
        hogares = tabla_deciles['hogares'].values
        ingresos_totales = ingresos.dot(hogares)
        hogares_totales = hogares.sum()

        # Calcular la fracción acumulada de ingresos y hogares para el Gini
        ingresos_acumulados = np.cumsum(ingresos * hogares) / ingresos_totales
        hogares_acumulados = np.cumsum(hogares) / hogares_totales

        # Calcular el área entre la curva de Lorenz y la línea de igualdad perfecta (Gini)
        gini = 1 - np.sum((hogares_acumulados[1:] - hogares_acumulados[:-1]) * (ingresos_acumulados[1:] + ingresos_acumulados[:-1]))

        # Almacenar los resultados para este estado y año
        resultados.append({
            'year': year,
            'estado': grupo['ent'].iloc[0],
            'ent': ent,
            'gini': gini,
            'ingresos_por_decil': ingresos_por_decil.to_dict()
        })

    # Convertir los resultados a un DataFrame para facilitar la visualización
    df_resultados = pd.DataFrame(resultados)
    return df_resultados

In [ ]:
def calcular_gini_y_deciles_mpio(df):
    # Lista para almacenar los resultados por entidad, municipio y año
    resultados = []

    # Agrupamos por 'year', 'entidad' y 'municipio'
    for (year, entidad, mpio), grupo in df.groupby(['year', 'entidad', 'mpio'], observed=True):

        # Calcular el total de hogares en el grupo
        total_hogares = grupo['factor'].sum()
        if total_hogares == 0:
            continue  # Omitir el grupo si no hay hogares

        # Ordenar el grupo por ingreso y calcular el tamaño del decil
        grupo = grupo.sort_values(by='ing_cor').reset_index(drop=True)
        tam_dec = int(total_hogares // 10)  # Tamaño del decil truncado

        # Sumar acumulativamente el factor para dividir en deciles
        grupo['ACUMULA'] = grupo['factor'].cumsum()
        grupo['DECIL'] = pd.cut(grupo['ACUMULA'], bins=[0] + [tam_dec * i for i in range(1, 11)], labels=range(1, 11), include_lowest=True)

        # Calcular el ingreso promedio y el número de hogares por decil, solo si el factor > 0
        ingresos_por_decil = grupo.groupby('DECIL', observed=True).apply(
            lambda x: np.average(x['ing_cor'], weights=x['factor']) if x['factor'].sum() > 0 else 0, include_groups=False
        )
        hogares_por_decil = grupo.groupby('DECIL', observed=True)['factor'].sum()

        # Crear tabla de deciles para calcular el Gini
        tabla_deciles = pd.DataFrame({
            'hogares': hogares_por_decil,
            'ingreso_promedio': ingresos_por_decil
        }).reset_index()

        # Calcular el Gini usando ingresos promedio por decil ponderados por el número de hogares
        ingresos = tabla_deciles['ingreso_promedio'].values
        hogares = tabla_deciles['hogares'].values
        ingresos_totales = ingresos.dot(hogares)
        hogares_totales = hogares.sum()

        # Calcular la fracción acumulada de ingresos y hogares para el Gini
        ingresos_acumulados = np.cumsum(ingresos * hogares) / ingresos_totales
        hogares_acumulados = np.cumsum(hogares) / hogares_totales

        # Calcular el área entre la curva de Lorenz y la línea de igualdad perfecta (Gini)
        gini = 1 - np.sum((hogares_acumulados[1:] - hogares_acumulados[:-1]) * (ingresos_acumulados[1:] + ingresos_acumulados[:-1]))

        # Almacenar los resultados para este municipio, entidad y año
        resultados.append({
            'year': year,
            'estado': grupo['entidad'].iloc[0],
            'ent': entidad,
            'mpio': mpio,
            'gini': gini,
            'ingresos_por_decil': ingresos_por_decil.to_dict()
        })

    # Convertir los resultados a un DataFrame para facilitar la visualización
    df_resultados = pd.DataFrame(resultados)
    return df_resultados

### Por estado

In [113]:
# Crear la columna entidad a partir de la columna ubica_geo, si no está presente
if 'ent' not in df_enigh.columns:
    df_enigh['ent'] = df_enigh['ubica_geo'].astype(str).str[:-3].astype(int)

In [115]:
# Calcular Gini y deciles por estado y año
df_resultados_ent = calcular_gini_y_deciles_ent(df_enigh)

# Mostrar los resultados
df_resultados_ent

,year,estado,ent,gini,ingresos_por_decil
0,2018,1,1,0.391868,"{1: 12897.004214827297, 2: 21632.406708729133,..."
1,2018,2,2,0.380719,"{1: 14237.357488951204, 2: 22686.515374712646,..."
2,2018,3,3,0.377081,"{1: 15551.894506027196, 2: 25963.08206786035, ..."
3,2018,4,4,0.429700,"{1: 8667.04770159055, 2: 14702.759441418617, 3..."
4,2018,5,5,0.391121,"{1: 12288.187593156617, 2: 20583.067962363028,..."
...,...,...,...,...,...
91,2022,28,28,0.374787,"{1: 14323.353483869223, 2: 24180.353357730688,..."
92,2022,29,29,0.336912,"{1: 13666.800018524662, 2: 20532.90198398626, ..."
93,2022,30,30,0.393564,"{1: 10013.872418311354, 2: 16677.85942718583, ..."
94,2022,31,31,0.390186,"{1: 14271.221113875514, 2: 22764.37680595659, ..."


In [116]:
# Ordenar el DataFrame por 'entidad' y 'year' para asegurar un cálculo secuencial
df_cambios_ent = df_resultados_ent.sort_values(by=['ent', 'year']).reset_index(drop=True)
# Calcular el ingreso promedio a partir de la columna 'ingresos_por_decil'
df_cambios_ent['promedio_ingresos'] = df_cambios_ent['ingresos_por_decil'].apply(lambda x: np.mean(list(x.values())))

# Calcular diferencias de ingresos promedio y Gini por entidad entre años
df_cambios_ent['diff_income_abs'] = df_cambios_ent.groupby('ent')['promedio_ingresos'].diff()
df_cambios_ent['diff_income_pct'] = df_cambios_ent.groupby('ent')['promedio_ingresos'].pct_change() * 100

df_cambios_ent['diff_gini_abs'] = df_cambios_ent.groupby('ent')['gini'].diff()
df_cambios_ent['diff_gini_pct'] = df_cambios_ent.groupby('ent')['gini'].pct_change() * 100

# Mostrar el DataFrame con los resultados
df_cambios_ent[['year', 'ent', 'promedio_ingresos', 'diff_income_abs', 'diff_income_pct', 'gini', 'diff_gini_abs', 'diff_gini_pct']]


,year,ent,promedio_ingresos,diff_income_abs,diff_income_pct,gini,diff_gini_abs,diff_gini_pct
0,2018,1,58691.265596,NaN,NaN,0.391868,NaN,NaN
1,2020,1,58205.951157,-485.314439,-0.826894,0.377191,-0.014677,-3.745331
2,2022,1,77725.786454,19519.835297,33.535807,0.401896,0.024704,6.549573
3,2018,2,59141.454197,NaN,NaN,0.380719,NaN,NaN
4,2020,2,66064.790280,6923.336083,11.706402,0.386445,0.005726,1.503905
...,...,...,...,...,...,...,...,...
91,2020,31,46034.006397,-2797.816804,-5.729495,0.431070,0.013741,3.292496
92,2022,31,61436.222910,15402.216513,33.458345,0.390186,-0.040884,-9.484318
93,2018,32,37938.907514,NaN,NaN,0.399324,NaN,NaN
94,2020,32,43034.172975,5095.265461,13.430185,0.410048,0.010723,2.685376


In [117]:
# Expandir la columna de 'ingresos_por_decil' en columnas separadas para cada decil
decil_ent_df = df_resultados_ent['ingresos_por_decil'].apply(pd.Series)
decil_ent_df.columns = [f'decil_{i}' for i in range(1, 11)]

# Concatenar el DataFrame original con las nuevas columnas de deciles
df_cambios_decil_ent = pd.concat([df_resultados_ent, decil_ent_df], axis=1)

# Ordenar el DataFrame por 'estado' y 'year' para garantizar un cálculo correcto
df_cambios_decil_ent = df_cambios_decil_ent.sort_values(by=['estado', 'year']).reset_index(drop=True)

# Calcular el crecimiento porcentual de ingresos por decil entre años para cada estado
for i in range(1, 11):
    df_cambios_decil_ent[f'decil_{i}_growth_pct'] = df_cambios_decil_ent.groupby('estado')[f'decil_{i}'].pct_change() * 100

#  Guardar en un nuevo DataFrame resultante
df_resultados_deciles_cambios_ent = df_cambios_decil_ent[['year', 'estado'] + [f'decil_{i}_growth_pct' for i in range(1, 11)]]
# Ver resultados
df_resultados_deciles_cambios_ent


,year,estado,decil_1_growth_pct,decil_2_growth_pct,decil_3_growth_pct,decil_4_growth_pct,decil_5_growth_pct,decil_6_growth_pct,decil_7_growth_pct,decil_8_growth_pct,decil_9_growth_pct,decil_10_growth_pct
0,2018,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020,1,0.509060,1.782509,3.540402,2.831463,1.216698,1.141688,1.872069,2.704859,1.870269,-7.261798
2,2022,1,36.902912,28.273465,23.035215,25.857224,27.003530,26.757608,26.504725,25.790592,30.638983,48.553602
3,2018,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,2,3.353774,10.321332,13.213493,13.489319,12.469313,11.332346,10.066129,8.188397,5.172443,17.768582
...,...,...,...,...,...,...,...,...,...,...,...,...
91,2020,31,-13.851597,-11.942341,-10.765464,-8.188979,-7.148542,-6.809502,-6.607201,-7.055280,-2.980001,-3.536370
92,2022,31,64.795950,53.786245,49.124245,46.344966,44.129855,41.327683,40.288598,37.376388,30.880640,20.054925
93,2018,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,2020,32,17.156553,10.790715,9.086506,8.728713,10.083737,10.664072,11.084936,12.337185,12.202244,18.238924


### Por municipio

In [118]:
# Crear la columna municipio a partir de la columna ubica_geo, si no está presente
if 'mpio' not in df_enigh.columns:
    df_enigh['mpio'] = df_enigh['ubica_geo'].astype(str).str[-3:].astype(int)

In [120]:
# Calcular Gini y deciles por entidad, municipio y año
df_resultados_mpio = calcular_gini_y_deciles_mpio(df_enigh)

# Mostrar los resultados
df_resultados_mpio

,year,estado,ent,mpio,gini,ingresos_por_decil
0,2018,1,1,1,0.384174,"{1: 13886.936196004206, 2: 23649.594432873273,..."
1,2018,1,1,2,0.317535,"{1: 9615.175025484199, 2: 14380.2775, 3: 18755..."
2,2018,1,1,3,0.364779,"{1: 8941.574878472222, 2: 14096.525753823744, ..."
3,2018,1,1,5,0.380003,"{1: 13869.776909224553, 2: 23571.796354356546,..."
4,2018,1,1,6,0.356462,"{1: 15250.188966613672, 2: 22961.815874476986,..."
...,...,...,...,...,...,...
3213,2022,32,32,53,0.322413,"{1: 13822.095191489361, 2: 19993.28598540146, ..."
3214,2022,32,32,54,0.278439,"{1: 9356.04211618257, 2: 12693.173631910428, 3..."
3215,2022,32,32,55,0.282837,"{1: 11293.602499999999, 2: 19762.247475728156,..."
3216,2022,32,32,56,0.397553,"{1: 14905.019361979166, 2: 26025.710069974553,..."


In [121]:
# Ordenar el DataFrame por 'entidad', 'municipio' y 'year' para asegurar un cálculo secuencial
df_cambios_mpio = df_resultados_mpio.sort_values(by=['ent', 'mpio', 'year']).reset_index(drop=True)
# Calcular el ingreso promedio a partir de la columna 'ingresos_por_decil'
df_cambios_mpio['promedio_ingresos'] = df_cambios_mpio['ingresos_por_decil'].apply(lambda x: np.mean(list(x.values())))

# Calcular diferencias de ingresos promedio y Gini por entidadd entre años
df_cambios_mpio['diff_income_abs'] = df_cambios_mpio.groupby('ent')['promedio_ingresos'].diff()
df_cambios_mpio['diff_income_pct'] = df_cambios_mpio.groupby('ent')['promedio_ingresos'].pct_change() * 100

df_cambios_mpio['diff_gini_abs'] = df_cambios_mpio.groupby('ent')['gini'].diff()
df_cambios_mpio['diff_gini_pct'] = df_cambios_mpio.groupby('ent')['gini'].pct_change() * 100

# Mostrar el DataFrame con los resultados
df_cambios_mpio[['year', 'ent', 'mpio', 'promedio_ingresos', 'diff_income_abs', 'diff_income_pct', 'gini', 'diff_gini_abs', 'diff_gini_pct']]

,year,ent,mpio,promedio_ingresos,diff_income_abs,diff_income_pct,gini,diff_gini_abs,diff_gini_pct
0,2018,1,1,62793.822912,NaN,NaN,0.384174,NaN,NaN
1,2020,1,1,60083.053331,-2710.769581,-4.316937,0.374823,-0.009350,-2.433814
2,2022,1,1,83914.645017,23831.591686,39.664415,0.398942,0.024118,6.434524
3,2018,1,2,33500.463425,-50414.181591,-60.077930,0.317535,-0.081406,-20.405519
4,2020,1,2,41580.055238,8079.591813,24.117851,0.323424,0.005888,1.854429
...,...,...,...,...,...,...,...,...,...
3213,2018,32,56,54539.625189,11026.988042,25.342036,0.348197,0.065360,23.108863
3214,2020,32,56,53860.300749,-679.324440,-1.245561,0.361787,0.013590,3.903029
3215,2022,32,56,68429.718970,14569.418221,27.050384,0.397553,0.035766,9.885919
3216,2020,32,57,34580.945713,-33848.773257,-49.465019,0.363697,-0.033856,-8.516168


In [122]:
# Expandir la columna de 'ingresos_por_decil' en columnas separadas para cada decil
decil_mpio_df = df_resultados_mpio['ingresos_por_decil'].apply(pd.Series)
decil_mpio_df.columns = [f'decil_{i}' for i in range(1, 11)]

# Concatenar el DataFrame original con las nuevas columnas de deciles
df_cambios_decil_mpio = pd.concat([df_resultados_mpio, decil_mpio_df], axis=1)

# Ordenar el DataFrame por 'entiddad', 'municipio' y 'year' para garantizar un cálculo correcto
df_cambios_decil_mpio = df_cambios_decil_mpio.sort_values(by=['ent', 'mpio', 'year']).reset_index(drop=True)

# Calcular el crecimiento porcentual de ingresos por decil entre años para cada municipio dentro de cada entidad
for i in range(1, 11):
    df_cambios_decil_mpio[f'decil_{i}_growth_pct'] = df_cambios_decil_mpio.groupby(['ent', 'mpio'])[f'decil_{i}'].pct_change(fill_method=None) * 100

# Guardar en un nuevo DataFrame resultante solo las columnas de crecimiento porcentual y detalles
df_resultados_deciles_cambios_mpio = df_cambios_decil_mpio[['year', 'mpio', 'ent'] + [f'decil_{i}_growth_pct' for i in range(1, 11)]]

# Ver resultados
df_resultados_deciles_cambios_mpio

,year,mpio,ent,decil_1_growth_pct,decil_2_growth_pct,decil_3_growth_pct,decil_4_growth_pct,decil_5_growth_pct,decil_6_growth_pct,decil_7_growth_pct,decil_8_growth_pct,decil_9_growth_pct,decil_10_growth_pct
0,2018,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020,1,1,-5.638397,-4.095661,-2.141105,-2.382958,-3.871767,-2.247289,-1.164170,-0.900185,-1.580586,-9.587139
2,2022,1,1,49.525687,34.652187,30.414315,32.030371,33.496024,31.886102,30.268086,29.890728,35.228819,57.183034
3,2018,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,2,1,15.191885,28.306688,20.168850,31.341543,25.275784,11.930041,19.594836,22.751371,25.489594,30.373649
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213,2018,56,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3214,2020,56,32,-22.959741,-9.006514,-4.680167,-3.162438,-5.424005,0.932478,0.649366,3.189665,10.225095,-5.874843
3215,2022,56,32,28.625866,25.038485,18.310205,19.203721,20.847872,11.482085,11.844718,17.019085,24.525358,51.733247
3216,2020,57,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# ENCO

In [123]:
df_enco

,fol,ent,con,v_sel,n_hog,h_mud,i_per,ing,mpio,ageb,...,p7,p8,p9,p10,p11,p12,p13,p14,p15,year
0,11B167,1,40006,3,1,0,1.0,1300.0,5,025-1,...,3,3,2,2,3,6,3,3,1,2018
1,11B167,1,40006,3,1,0,<NA>,NaN,5,025-1,...,3,3,2,2,3,6,3,3,1,2018
2,11B167,1,40006,3,1,0,<NA>,NaN,5,025-1,...,3,3,2,2,3,6,3,3,1,2018
3,11B167,1,40006,3,1,0,<NA>,NaN,5,025-1,...,3,3,2,2,3,6,3,3,1,2018
4,11B167,1,40006,3,1,0,1.0,1100.0,5,025-1,...,3,3,2,2,3,6,3,3,1,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217135,12B212,32,40385,4,1,0,,NaN,56,037-9,...,3,3,2,2,2,6,3,3,3,2022
217136,12B212,32,40385,4,1,0,,NaN,56,037-9,...,3,3,2,2,2,6,3,3,3,2022
217137,12B212,32,40385,4,1,0,2,4000.0,56,037-9,...,3,3,2,2,2,6,3,3,3,2022
217138,12B212,32,40385,3,1,0,,NaN,56,037-9,...,1,1,1,1,2,2,2,3,2,2022


In [124]:
# Lista de preguntas
preguntas = [f'p{i}' for i in range(1, 16)]  # P1, P2, ..., P15

# Crear un DataFrame vacío para almacenar resultados
resultados_porcentajes = pd.DataFrame()

# Calcular porcentajes por año, pregunta y respuesta
for pregunta in preguntas:
    # Agrupar por año y respuesta, calcular el porcentaje
    frecuencias = df_enco.groupby(['year', pregunta]).size() / df_enco.groupby('year').size() * 100
    frecuencias_df = frecuencias.reset_index()
    frecuencias_df.columns = ['Año', 'Respuesta', 'Porcentaje']
    frecuencias_df['Pregunta'] = pregunta

    # Agregar al DataFrame general
    resultados_porcentajes = pd.concat([resultados_porcentajes, frecuencias_df], ignore_index=True)

# Reorganizar las columnas
resultados_porcentajes = resultados_porcentajes[['Pregunta', 'Año', 'Respuesta', 'Porcentaje']]

# Mostrar resultados
resultados_porcentajes

,Pregunta,Año,Respuesta,Porcentaje
0,p1,2018,1,0.829269
1,p1,2018,2,14.289585
2,p1,2018,3,48.922185
3,p1,2018,4,33.004688
4,p1,2018,5,2.944849
...,...,...,...,...
229,p15,2020,4,0.116124
230,p15,2022,1,11.284858
231,p15,2022,2,9.933356
232,p15,2022,3,78.645271


In [125]:

# Crear un DataFrame vacío para almacenar resultados
resultados_estado_porcentajes = pd.DataFrame()

# Calcular porcentajes por año, estado, pregunta y respuesta
for pregunta in preguntas:
    # Agrupar por año, estado y respuesta, calcular el porcentaje
    frecuencias_ent = df_enco.groupby(['year', 'ent', pregunta]).size() / df_enco.groupby(['year', 'ent']).size() * 100
    frecuencias_ent_df = frecuencias_ent.reset_index()
    frecuencias_ent_df.columns = ['Año', 'ent', 'Respuesta', 'Porcentaje']
    frecuencias_ent_df['Pregunta'] = pregunta

    # Agregar al DataFrame general
    resultados_estado_porcentajes = pd.concat([resultados_estado_porcentajes, frecuencias_ent_df], ignore_index=True)

# Reorganizar las columnas
resultados_estado_porcentajes = resultados_estado_porcentajes[['Pregunta', 'Año', 'ent', 'Respuesta', 'Porcentaje']]

# Mostrar resultados
resultados_estado_porcentajes

,Pregunta,Año,ent,Respuesta,Porcentaje
0,p1,2018,1,1,0.580616
1,p1,2018,1,2,12.237606
2,p1,2018,1,3,55.247879
3,p1,2018,1,4,31.353283
4,p1,2018,1,5,0.580616
...,...,...,...,...,...
6497,p15,2022,31,4,0.927152
6498,p15,2022,32,1,21.300597
6499,p15,2022,32,2,6.502986
6500,p15,2022,32,3,71.333776


In [126]:
# Crear un DataFrame vacío para almacenar resultados
resultados_mpio_porcentajes = pd.DataFrame()

# Calcular porcentajes por estado, municipio, pregunta y respuesta
for pregunta in preguntas:
    # Agrupar por estado, municipio y respuesta, calcular el porcentaje
    frecuencias_mpio = df_enco.groupby(['year', 'ent', 'mpio', pregunta]).size() / df_enco.groupby(['year','ent', 'mpio']).size() * 100
    frecuencias_mpio_df = frecuencias_mpio.reset_index()
    frecuencias_mpio_df.columns = ['Año', 'Estado', 'mpio', 'Respuesta', 'Porcentaje']
    frecuencias_mpio_df['Pregunta'] = pregunta

    # Agregar al DataFrame general
    resultados_mpio_porcentajes = pd.concat([resultados_mpio_porcentajes, frecuencias_mpio_df], ignore_index=True)

# Reorganizar las columnas
resultados_mpio_porcentajes = resultados_mpio_porcentajes[['Año', 'Pregunta', 'Estado', 'mpio', 'Respuesta', 'Porcentaje']]

# Mostrar resultados
resultados_mpio_porcentajes

,Año,Pregunta,Estado,mpio,Respuesta,Porcentaje
0,2018,p1,1,1,1,0.613208
1,2018,p1,1,1,2,12.547170
2,2018,p1,1,1,3,54.575472
3,2018,p1,1,1,4,31.650943
4,2018,p1,1,1,5,0.613208
...,...,...,...,...,...,...
23325,2022,p15,32,17,4,1.026958
23326,2022,p15,32,56,1,19.780220
23327,2022,p15,32,56,2,3.846154
23328,2022,p15,32,56,3,75.686813


# Censo

In [127]:
df_cve_ent = pd.read_csv(censo_ent_path, encoding='utf-8')
df_cve_mun = pd.read_csv(censo_mun_path, encoding='utf-8')

# Map

## Geopandas

In [128]:
gdf_ent['cvegeo']=gdf_ent['cvegeo'].astype(int)
gdf_mun['cvegeo']=gdf_mun['cvegeo'].astype(int)

gdf_mun merge with gdf_mun_pob and 

In [129]:
gdf_mun_pob = gdf_mun.merge(df_cve_mun, on=['cvegeo'], how='left')
gdf_mun_pob=gdf_mun_pob[['cvegeo','nom_geo','pob_tot','rel_h_m','geometry']] # desired order

In [130]:
gdf_ent_pob = gdf_ent.merge(df_cve_ent, on=['cvegeo'], how='left')
gdf_ent_pob=gdf_ent_pob[['cvegeo','nom_geo','pob_tot','rel_h_m','geometry']] # desired order

## Maps

In [131]:
# Define the style function to color the polygons based on rel_h_m values
def style_function(feature):
    # Convert rel_h_m to numeric, coerce errors to NaN
    rel_h_m_value = pd.to_numeric(feature['properties']['rel_h_m'], errors='coerce')

    if pd.notna(rel_h_m_value):  # Check if the value is not NaN
        if 99 <= rel_h_m_value <= 101:
            fill_color = 'green'
        elif rel_h_m_value > 101:
            fill_color = 'blue'
        else:
            fill_color = 'pink'
    else:
        fill_color = 'gray'  # Default color for NaN values

    return {
        'fillColor': fill_color,
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.5,
    }

In [132]:
merged_gdf_ent = pd.merge(gdf_ent_pob, df_cambios_ent, left_on='cvegeo', right_on='ent')

In [ ]:
# Create a folium map
m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)  # Center on Mexico

# Define styles for each year to make the map visually clear
year_styles = {
    2018: {'fillColor': '#1f77b4', 'color': 'black', 'weight': 1, 'fillOpacity': 0.6},  # Blue
    2020: {'fillColor': '#ff7f0e', 'color': 'black', 'weight': 1, 'fillOpacity': 0.6},  # Orange
    2022: {'fillColor': '#2ca02c', 'color': 'black', 'weight': 1, 'fillOpacity': 0.6},  # Green
}

# Function to style polygons based on the selected year
def style_function(feature, year):
    return year_styles[year]

# Add a descriptive title as an HTML marker
title_html = '''
    <div style="position: fixed; 
                top: 10px; left: 50px; width: 300px; height: 80px; 
                background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
                padding: 10px;">
        <b>Interactive Map:</b> Select a year to view data for that period (untoggle and toggle if necesary).
    </div>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Add a layer for each year
for year in [2018, 2020, 2022]:
    year_gdf = merged_gdf_ent[merged_gdf_ent['year'] == year]
    
    # Add GeoJson for this year's data
    folium.GeoJson(
        year_gdf,
        style_function=lambda feature, year=year: style_function(feature, year),  # Year-specific style
        tooltip=folium.GeoJsonTooltip(
            fields=['nom_geo', 'pob_tot', 'rel_h_m', 'gini'],
            aliases=['Entity:', 'Population:', 'Male per 100 Female:', f'Gini ({year}):'],
            localize=True
        ),
        name=f"Year {year}",  # Layer name for this year
        control=True
    ).add_to(m)

# Add layer control to enforce single selection
folium.LayerControl(collapsed=False, autoZIndex=True).add_to(m)

# Add a legend to explain the colors
legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 200px; height: 90px; 
                background-color: white; border:2px solid grey; z-index:9999; font-size:12px;
                padding: 10px;">
        <b>Legend:</b><br>
        <i style="background: #1f77b4; width: 10px; height: 10px; float: left; margin-right: 10px;"></i> 2018<br>
        <i style="background: #ff7f0e; width: 10px; height: 10px; float: left; margin-right: 10px;"></i> 2020<br>
        <i style="background: #2ca02c; width: 10px; height: 10px; float: left; margin-right: 10px;"></i> 2022<br>
    </div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Display or save the map
m.save('interactive_ent_map.html') # Uncomment to save the map
#m #Uncomment to see in notebook

In [134]:
df_cambios_mpio['cvegeo']=df_cambios_mpio['ent']*1000+df_cambios_mpio['mpio']

In [135]:
merged_gdf_mun = pd.merge(gdf_mun_pob, df_cambios_mpio, on='cvegeo')

Disclaimer: The folium map for municipality it's to heavy to push to github, so one can run it in their own system with the following code:

Disclaimer 2: Not all municipalities have a gini value for every year, or even for any year

In [ ]:
# Create a folium map
m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)  # Center on Mexico

# Define styles for each year to make the map visually clear
year_styles = {
    2018: {'fillColor': '#1f77b4', 'color': 'black', 'weight': 1, 'fillOpacity': 0.6},  # Blue
    2020: {'fillColor': '#ff7f0e', 'color': 'black', 'weight': 1, 'fillOpacity': 0.6},  # Orange
    2022: {'fillColor': '#2ca02c', 'color': 'black', 'weight': 1, 'fillOpacity': 0.6},  # Green
}

# Function to style polygons based on the selected year
def style_function(feature, year):
    return year_styles[year]

# Add a descriptive title as an HTML marker
title_html = '''
    <div style="position: fixed; 
                top: 10px; left: 50px; width: 300px; height: 80px; 
                background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
                padding: 10px;">
        <b>Interactive Map:</b> Select a year to view data for that period (untoggle and toggle if necesary).
    </div>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Add a layer for each year
for year in [2018, 2020, 2022]:
    year_gdf = merged_gdf_mun[merged_gdf_mun['year'] == year]
    
    # Add GeoJson for this year's data
    folium.GeoJson(
        year_gdf,
        style_function=lambda feature, year=year: style_function(feature, year),  # Year-specific style
        tooltip=folium.GeoJsonTooltip(
            fields=['nom_geo', 'pob_tot', 'rel_h_m', 'gini'],
            aliases=['Municipality:', 'Population:', 'Male per 100 Female:', f'Gini ({year}):'],
            localize=True
        ),
        name=f"Year {year}",  # Layer name for this year
        control=True
    ).add_to(m)

# Add layer control to enforce single selection
folium.LayerControl(collapsed=False, autoZIndex=True).add_to(m)

# Add a legend to explain the colors
legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 200px; height: 90px; 
                background-color: white; border:2px solid grey; z-index:9999; font-size:12px;
                padding: 10px;">
        <b>Legend:</b><br>
        <i style="background: #1f77b4; width: 10px; height: 10px; float: left; margin-right: 10px;"></i> 2018<br>
        <i style="background: #ff7f0e; width: 10px; height: 10px; float: left; margin-right: 10px;"></i> 2020<br>
        <i style="background: #2ca02c; width: 10px; height: 10px; float: left; margin-right: 10px;"></i> 2022<br>
    </div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Display or save the map
#m.save('interactive_mun_map.html') # Uncomment to save the map
#m #Uncomment to see in notebook